In [1]:
import numpy as np 
import pandas as pd 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy import sparse
import random
!pip install lightfm
import lightfm 
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
drive.mount("/content/drive")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 14.2 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-linux_x86_64.whl size=747415 sha256=d313322685cb5b12685b9fd41c893acb7b3e241a466be9c112ba1e0e4a89d3bd
  Stored in directory: /root/.cache/pip/wheels/ec/bb/51/9c487d021c1373b691d13cadca0b65b6852627b1f3f43550fa
Successfully built lightfm
Mounted at /content/drive


In [2]:
# import pandas as pd
# df=pd.read_csv('/content/drive/MyDrive/CMPE256/data_csv/mpd.slice.0-999.csv')
# df.head(5)
import glob
import os

path = r'/content/drive/MyDrive/CMPE256/data_csv/10slides' # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))

df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)


/usr/local/lib/python3.8/dist-packages/pandas/core/reshape/concat.py:348: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  objs = list(objs)


In [3]:
df

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,playlist_pid,name,collaborative,modified_at,num_tracks,num_albums,num_followers,num_edits,duration_ms_playlist,num_artists,description
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,0,Throwbacks,False,1493424000,52,47,1,6,11532414,37,NaN
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,0,Throwbacks,False,1493424000,52,47,1,6,11532414,37,NaN
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),0,Throwbacks,False,1493424000,52,47,1,6,11532414,37,NaN
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,0,Throwbacks,False,1493424000,52,47,1,6,11532414,37,NaN
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,0,Throwbacks,False,1493424000,52,47,1,6,11532414,37,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663897,21,Adventure Club,spotify:track:296fTW1CMX9j9mBRXKQMYi,spotify:artist:5CdJjUi9f0cVgo9nFuJrFa,Thunderclap,spotify:album:2DVyzKYzqlQo4uTJLVtvBt,273949,Calling All Heroes - EP,8999,groovin,False,1465948800,26,18,1,4,7001698,10,NaN
663898,22,Adventure Club,spotify:track:5yQmKyKEdjtxFDPIK9mFLn,spotify:artist:5CdJjUi9f0cVgo9nFuJrFa,Rise & Fall (feat. Krewella),spotify:album:5xkfX4DNx7hZYrGy8H1YUJ,361875,Rise & Fall (feat. Krewella),8999,groovin,False,1465948800,26,18,1,4,7001698,10,NaN
663899,23,Adventure Club,spotify:track:6D1kEYqdY4D3okAOfeKouI,spotify:artist:5CdJjUi9f0cVgo9nFuJrFa,Limitless (feat. Delaney Jane),spotify:album:45hTiiH7JpJx1P5TN6iDDG,244610,Red // Blue,8999,groovin,False,1465948800,26,18,1,4,7001698,10,NaN
663900,24,Adventure Club,spotify:track:3NAGKEkJN7cUTaIYqFnF8l,spotify:artist:5CdJjUi9f0cVgo9nFuJrFa,Crash 2.0 (Adventure Club vs. DallasK),spotify:album:45hTiiH7JpJx1P5TN6iDDG,254879,Red // Blue,8999,groovin,False,1465948800,26,18,1,4,7001698,10,NaN


In [4]:
df.isna().sum()

pos                          0
artist_name                  0
track_uri                    0
artist_uri                   0
track_name                   0
album_uri                    0
duration_ms                  0
album_name                   0
playlist_pid                 0
name                         0
collaborative                0
modified_at                  0
num_tracks                   0
num_albums                   0
num_followers                0
num_edits                    0
duration_ms_playlist         0
num_artists                  0
description             649904
dtype: int64

In [5]:
df['track_uri'].isna().sum()

0

In [6]:
df.drop_duplicates(subset = 'track_uri')['track_name'].value_counts()

Home                                       89
Silent Night                               88
Intro                                      66
O Holy Night                               63
You                                        61
                                           ..
Early In The Morning, I'll Come Calling     1
Down The Burning Ropes                      1
Follow You Down To The Red Oak Tree         1
Breaking Hearts                             1
Dynasty - Dada Life Remix                   1
Name: track_name, Length: 132776, dtype: int64

In [7]:
df.to_csv('10kclean.csv', encoding='utf-8', index=False)

In [8]:
#Creating matrix 
n_pls = df.playlist_pid.unique().shape[0]
n_songs = df.track_uri.unique().shape[0]

print(f'Num playlists: {n_pls}')
print(f'Num unique songs: {n_songs}')
print(f'Sparsity: {100 - float(df.shape[0]) /float(n_pls*n_songs) * 100:0.3f}%')

Num playlists: 10000
Num unique songs: 169613
Sparsity: 99.961%


In [9]:
cols = df.playlist_pid.astype('category').cat.codes 
rows = df.track_uri.astype('category').cat.codes

In [10]:
cols.unique().shape

(10000,)

In [11]:
likes = np.ones(df.shape[0]) #1s if song is in playlist
likes = sparse.csr_matrix((likes, (rows, cols)), dtype=np.float64) #make sparse matrix

In [12]:
print(df.groupby(by=['playlist_pid', 'track_uri']).count()['track_name'].shape)

(655479,)


In [13]:
print(f'min num songs in playlist: {likes.sum(axis=1).min()}')
print(f'min num playlists song is featured in: {likes.sum(axis=0).min()}')

min num songs in playlist: 1.0
min num playlists song is featured in: 5.0


In [14]:
likes = likes.tocsc()

In [15]:
def train_test_split(likes, num_to_mask, rd_seed=random.seed(0)):
  '''
  Make training set with masked interactions

  likes: the full sparse matrix of playlist-song interactions
  num_to_mask: how many interactions we will mask for our training set
  rd_seed: random.seed to use, default 0

  Inspired by: https://www.ethanrosenthal.com/2016/10/19/implicit-mf-part-1/
  '''
  #make the train and test matrices
  #use lil matrix for efficiency
  train = likes.copy().tolil() #copy all the data to train for later masking
  test = likes.copy().tolil()
  #test = sparse.lil_matrix(likes.shape, dtype=np.float64) #initialize an empty matrix

  for pl in np.arange(0, likes.shape[1]): #for each playlist
    if len(likes.getcol(pl).indices) >= 20: #playlist must have 20+ unique songs
      #mask a random subset of 10 'likes' (1s)
      masked_likes = np.random.choice(likes.getcol(pl).indices, 
                                        size=num_to_mask, 
                                        replace=False)
      train[masked_likes, pl] = 0.

      #mask everything but these likes in the test set
      #scipy has shape assignment mismatch when trying to assign column values
      #from likes matrix to test matrix, even though shapes are equivalent
      cells_to_mask = likes.getcol(pl).indices[~np.isin(likes.getcol(pl).indices, masked_likes)]
      test[cells_to_mask, pl] = 0.

    else: #mask everything from the test set if playlist is under 20 unique songs
      cells_to_mask = likes.getcol(pl).indices
      test[cells_to_mask, pl] = 0.

  return train.tocsc(), test.tocsc()

In [16]:
train_set, val_set = train_test_split(likes, 10)

In [17]:
train_set, val_set

(<169613x10000 sparse matrix of type '<class 'numpy.float64'>'
 	with 571719 stored elements in Compressed Sparse Column format>,
 <169613x10000 sparse matrix of type '<class 'numpy.float64'>'
 	with 83760 stored elements in Compressed Sparse Column format>)

In [18]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 8.7 MB/s 


In [19]:
import implicit

In [20]:
train_set

<169613x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 571719 stored elements in Compressed Sparse Column format>

In [21]:
alpha = 10
conf_train_set = (train_set*alpha).tocsr()
conf_train_set.dtype

dtype('float64')

In [135]:
#initialize model
!export OPENBLAS_NUM_THREADS=1
mf_model = implicit.als.AlternatingLeastSquares(factors=32, iterations=50)

In [137]:
#make pl_index - filter out playlists that did not get any values in the test set
def make_pl_index(train_set):
  '''
  returns array of indiczes of playlists that have interactions in the test set
  '''
  pl_index = []
  for pl in np.arange(0, train_set.shape[1]): #for each playlist
    if len(train_set.getcol(pl).indices) >= 10: #playlist in train_set must have 10+ songs remaining
      pl_index.append(pl)

  return np.array(pl_index)

In [138]:
def R_precision(recs, mat, pl_index):
  precision = []
  for pl in pl_index:
    top_k_recs = np.array(recs)[:, :500] 
    relevant_songs = mat.getcol(pl).indices
    precision.append(float(len(np.intersect1d(top_k_recs, relevant_songs)))/relevant_songs.shape[0])

  return np.mean(precision)

In [27]:
pl_index = make_pl_index(train_set)
pl_index.shape

(9666,)

In [28]:
train_set.tocsr()

<169613x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 571719 stored elements in Compressed Sparse Row format>

In [140]:
result=[]
for j in [10,20,30]:
  mf_model = implicit.als.AlternatingLeastSquares(factors=j, iterations=50)
  mf_model.fit(conf_train_set)
  recs_train=[]
  for i in range(500):
    recs_train.append(mf_model.recommend(i,train_set.tocsr()[i]))
  mean_precision = R_precision(recs_train, train_set, pl_index)
  result.append({'Nfactor':j,'r-precision':mean_precision})
print(result)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

[{'Nfactor': 10, 'r-precision': 0.0026968753040653593}, {'Nfactor': 20, 'r-precision': 0.004173398578845105}, {'Nfactor': 30, 'r-precision': 0.004625460690370349}]


LightFM

In [99]:
df.artist_uri.value_counts()

spotify:artist:3TVXtAsR1Inumwj472S9r4    8341
spotify:artist:5K4W6rqBFWDnAN6FQUkS6x    3837
spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg    3353
spotify:artist:5pKCCKE2ajJHZ9KAiaK11H    3141
spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ    2939
                                         ... 
spotify:artist:2zwTFhb1SlPSuER7CMNOiV       1
spotify:artist:0AOY0FBz2frjClgCdTWOlD       1
spotify:artist:1Pn6pKlgzxcH6iIRp08dQr       1
spotify:artist:34hNaRFSpaudofxFZMmfY5       1
spotify:artist:2PmIyxmEFsNfQJjEifPDZC       1
Name: artist_uri, Length: 35862, dtype: int64

In [100]:
n_artist = df.artist_uri.unique().shape[0]
print(f'Num unique artists: {n_artist}')

Num unique artists: 35862


In [101]:
n_artist_name=df.artist_name.unique().shape[0]
print(f'Num unique artists: {n_artist}')

Num unique artists: 35862


In [102]:
import numpy as np 
import pandas as pd 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy import sparse
import random
!pip install lightfm
import lightfm 
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [103]:
df_playlist=df
df_playlist.columns = df_playlist.columns.str.replace('"', '')
# df_playlist.columns = df_playlist.columns.str.replace('name', '')
df_playlist.columns = df_playlist.columns.str.replace(' ', '')
df_playlist.columns

Index(['pos', 'artist_name', 'track_uri', 'artist_uri', 'track_name',
       'album_uri', 'duration_ms', 'album_name', 'playlist_pid', 'name',
       'collaborative', 'modified_at', 'num_tracks', 'num_albums',
       'num_followers', 'num_edits', 'duration_ms_playlist', 'num_artists',
       'description'],
      dtype='object')

In [104]:
df_playlist = df_playlist.groupby('artist_name').filter(lambda x : len(x)>=50)

In [105]:
df_playlist = df_playlist[df_playlist.groupby('playlist_pid').artist_name.transform('nunique') >= 10]

In [142]:
size = lambda x: len(x)
df_freq = df_playlist.groupby(['playlist_pid', 'artist_name','track_uri']).agg('size').reset_index().rename(columns={0:'freq'})[['playlist_pid', 'artist_name', 'track_uri','freq']].sort_values(['freq'], ascending=False)
df_freq.head()

,playlist_pid,artist_name,track_uri,freq
99539,3158,Marvin Gaye,spotify:track:6c01x4nc6S52tIKjC5TKh7,12
99514,3158,Marvin Gaye,spotify:track:27hmFto0tB3dAcNWOCqyhQ,8
411250,11015,Twenty One Pilots,spotify:track:19W5OTEcQI3ZoRW1HERMyy,8
329136,8147,Lifehouse,spotify:track:7L605WhF5EGf34ggj87yK6,7
418357,11195,Twenty One Pilots,spotify:track:6i0V12jOa3mr6uu4WYhUBr,7


In [173]:
df_track = pd.DataFrame(df_freq["artist_name"].unique())
df_track = df_track.reset_index()
df_track = df_track.rename(columns={'index':'artist_id', 0:'artist_name'})
df_freq  = pd.merge(df_freq ,df_track, how='inner', on='artist_name')

In [174]:
df_freq

,playlist_pid,artist_name,track_uri_x,freq,artist_id_x,track_uri_y,artist_id_y
0,3158,Marvin Gaye,spotify:track:6c01x4nc6S52tIKjC5TKh7,12,0,0,0
1,3158,Marvin Gaye,spotify:track:27hmFto0tB3dAcNWOCqyhQ,8,0,0,0
2,6288,Marvin Gaye,spotify:track:2H3ZUSE54pST4ubRd5FzFR,3,0,0,0
3,3158,Marvin Gaye,spotify:track:1CAbtyVgGvdW3dhNejIHlS,3,0,0,0
4,3158,Marvin Gaye,spotify:track:2HEbCul29U674Gml0Zf0DU,3,0,0,0
...,...,...,...,...,...,...,...
455073,5269,Hope College Worship,spotify:track:0yZDfW62GzjHlkL4sdoUmV,1,2072,2072,2072
455074,5269,Hope College Worship,spotify:track:0xOtEP3RAjd2PKYBouRWaC,1,2072,2072,2072
455075,5269,Hope College Worship,spotify:track:0rwVoCzT7vEN2TavpTIhdM,1,2072,2072,2072
455076,5229,Los Temerarios,spotify:track:25xeGvjWvTBu8RkEF8kRQr,1,2073,2073,2073


In [175]:
df_track.shape

(2075, 2)

In [176]:
# def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
#     '''
#     Function to create an interaction matrix dataframe from transactional type interactions
#     Required Input -
#         - df = Pandas DataFrame containing user-item interactions
#         - user_col = column name containing user's identifier
#         - item_col = column name containing item's identifier
#         - rating col = column name containing user feedback on interaction with a given item
#         - norm (optional) = True if a normalization of ratings is needed
#         - threshold (required if norm = True) = value above which the rating is favorable
#     Expected output - 
#         - Pandas dataframe with user-item interactions ready to be fed in a recommendation algorithm
#     '''
#     interactions = df.groupby([user_col, item_col])[rating_col] \
#             .sum().unstack().reset_index(). \
#             fillna(0).set_index(user_col)
#     if norm:
#         interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
#     return interactions

In [187]:
for i in [0.01,0.05,0.1]:
  model = LightFM(learning_rate=i, loss='bpr')
  model.fit(train_set, epochs=10)
  train_precision = precision_at_k(model, train_set, k=10).mean()
  train_auc = auc_score(model, train_set).mean()
  test_auc = auc_score(model, val_set).mean()
  print('learning rate: %.2f'%(i))
  print('Precision: train %.2f' % (train_precision))
  print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))


learning rate: 0.01
Precision: train 0.00
AUC: train 0.51, test 0.49.
learning rate: 0.05
Precision: train 0.03
AUC: train 0.91, test 0.77.
learning rate: 0.10
Precision: train 0.06
AUC: train 0.97, test 0.83.
